In [1]:
import os
import time
from datetime import date
from dateutil.relativedelta import relativedelta

from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException, InvalidSessionIdException, StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager


class Browser:
    def __init__(self):
        self.servico = Service(ChromeDriverManager().install())
        options = webdriver.ChromeOptions()
        options.headless = True
        self._driver = self.retornar_browser()

    def retornar_browser(self):
        return webdriver.Chrome(service=self.servico)

    def acessar_site(self, link):
        self._driver.get(link)

    def encontre_by_xpath(self, xpath):
        elemento = self._driver.find_element(By.XPATH, xpath)
        return elemento

    def encontre_elements_by_class(self, class_name):
        elementos = self._driver.find_elements(By.CLASS_NAME, class_name)
        return elementos

    @staticmethod
    def click_elemento(elemento):
        elemento.click()

    @staticmethod
    def send_keys(elemento, string):
        elemento.send_keys(string)

    def click_home_key(self):
        self._driver.find_element(By.XPATH, '//body').send_keys(Keys.HOME)

    def retorna_html_page(self):
        return self._driver.page_source

    def retornar_todas_as_abas_abertas(self):
        return self._driver.window_handles

    def mudar_aba_browser(self, numero_aba):
        abas = self.retornar_todas_as_abas_abertas()
        self._driver.switch_to.window(abas[numero_aba])

    @staticmethod
    def send_enter_key(elemento):
        elemento.send_keys(Keys.ENTER)

    def click_end_key(self):
        self._driver.find_element(By.XPATH, '//body').send_keys(Keys.END)

    def find_element_xpath_explicity_wait_visibility(self, tempo, xpath):
        elemento = WebDriverWait(self._driver, tempo).until(EC.visibility_of_element_located((By.XPATH, xpath)))
        return elemento

    def find_element_class_name_explicity_wait_visibility(self, tempo, class_name):
        elemento = WebDriverWait(self._driver, tempo).until(EC.visibility_of_element_located((By.CLASS_NAME, class_name)))
        return elemento

    def find_element_xpath_explicity_wait_clickable(self, tempo, xpath):
        # wait = WebDriverWait(self._driver, tempo)
        element = WebDriverWait(self._driver, tempo).until(EC.element_to_be_clickable((By.XPATH, xpath)))
        return element

    def find_element_xpath_explicity_wait_element_to_be_selected(self, tempo, xpath):
        # wait = WebDriverWait(self._driver, tempo)
        element = WebDriverWait(self._driver, tempo).until(EC.element_to_be_selected((By.XPATH, xpath)))
        return element
    

# Clase que contém o itinerário de voos
class Itinerario:
    def __init__(self):
        self._lista_aeroporto = self.lista_aeroporto()

    # Retornar o itinerario a ser usado pelo browser
    def retorna_itinerario(self):
        for ida in self._lista_aeroporto:
            for volta in self._lista_aeroporto:
                if ida == volta:
                    pass
                else:
                    yield ida, volta

    # função que retornar a lista com os códigos dos aeroportos
    @staticmethod
    def lista_aeroporto():
        lista_aeroporto = [
            "FORTALEZA",
            "VIX",
            "BELEM",
            "SALVADOR da Bahia",
            "POA",
            "Rio Branco",
            "ARACAJU",
            "CNF",
            "GRU",
            "CWB",
            "BSB",
            "SLZ",
            "CGR",
            "GYN",
            "RECIFE",
            "GIG",
        ]
        return lista_aeroporto

    @staticmethod
    def dict_dias_semana(dia_da_semana):
        semena_dias = ("Segunda-feira", "Terça-feira", "Quarta-feira", "Quinta-feira", "Sexta-feira", "Sábado", "Domingo")
        return semena_dias[dia_da_semana]


# Classe que contém as ações no site da GOL
class Latan(Browser, Itinerario):
    def __init__(self):
        # self._driver = self.retornar_browser(servico)
        # Link utilizado para acessar site da GOL
        self._url = 'https://www.latamairlines.com/br/pt'

        # Data atual
        self._data_atual = self.calcula_data_atual()
        self._data_ida = self.calcula_data_ida()
        self._data_volta = self.calcula_data_volta()

        self.data_atual_formatada = self._data_atual.strftime("%d/%m/%Y")
        self._data_ida_formatada = self._data_ida.strftime("%d/%m/%Y")
        self._data_volta_formatada = self._data_volta.strftime("%d/%m/%Y")

        self.coluna_nomes = pd.MultiIndex.from_tuples([('ORIGEM', f'Data Partida: {self._data_ida_formatada}'), ('DESTINO', f'Data Retorno: {self._data_volta_formatada}'), ('PREÇO DA PASSAGEM', f'Dia Coleta: {self.data_atual_formatada}')])
        self.coluna_nomes_volta = pd.MultiIndex.from_tuples([('DESTINO', f'Data Retorno: {self._data_volta_formatada}'), ('ORIGEM', f'Data Partida: {self._data_ida_formatada}'), ('PREÇO DA PASSAGEM', f'Dia Coleta: {self.data_atual_formatada}')])
        # Df que guarda os itinerarios de origem
        self.df_ida_origem = pd.DataFrame(columns=self.coluna_nomes)

        # Df que guar os itinerários de volta
        self.df_volta_destino = pd.DataFrame(columns=self.coluna_nomes_volta)

        # Df que guarda os itinerários de ida e volta
        self.df_preco_ida_volta = pd.DataFrame(columns=self.coluna_nomes)

        # Chamar super class Browser
        # super(Browser, self).__init__()

        # Chamar super class Itineario
        # super(Itinerario, self).__init__()
        Itinerario.__init__(self)

    # Método que realiza os processos do scraping
    def scraping(self):
        comeco = time.time()
        data_atual = self._data_atual.strftime("%d_%m_%Y")
        try:
            os.mkdir(f'Extração_LATAN_{data_atual}')
        except FileExistsError:
            pass
        for origem, destino in self.retorna_itinerario():
            while True:
                try:
                    print(f'Voos de {origem} para {destino}')
                    Browser.__init__(self)
                    # Acessar site
                    self.acessar_site(self._url)

                    # para ceitar cookies do site da latan
                    tempo = 15
                    try:
                        xpath = '//*[@id="cookies-politics-button"]'
                        element_cookie = self.find_element_xpath_explicity_wait_clickable(tempo, xpath)
                        self.click_elemento(element_cookie)
                        print('Cookies aceitos!')

                    except TimeoutException:
                        print('Sem cookies dessa vez!')

                    # Para escrever o aeroporot de origem
                    xpath = '//*[@id="txtInputOrigin_field"]'
                    elemento_input_origem = self.find_element_xpath_explicity_wait_visibility(tempo, xpath)
                    self.click_elemento(elemento_input_origem)
                    time.sleep(0.5)
                    self.send_keys(elemento_input_origem, origem)

                    # Para selecionar o aeroporto de origem
                    xpath = '//*[@id="popperExtended"]'
                    elemento_escolher_origem = self.find_element_xpath_explicity_wait_clickable(tempo, xpath)
                    self.click_elemento(elemento_escolher_origem)

                    # Para escrever o aeroporot de destino
                    xpath = '//*[@id="txtInputDestination_field"]'
                    elemento_input_origem = self.find_element_xpath_explicity_wait_clickable(tempo, xpath)
                    self.click_elemento(elemento_input_origem)
                    time.sleep(0.5)
                    self.send_keys(elemento_input_origem, destino)

                    # Para selecionar o aeroporto de destino
                    xpath = '//*[@id="popperExtended"]'
                    elemento_escolher_origem = self.find_element_xpath_explicity_wait_clickable(tempo, xpath)
                    self.click_elemento(elemento_escolher_origem)

                    # Para clicar no label de dia de partida
                    xpath = '//*[@id="departureDate"]'
                    elemento_label_data_partida = self.find_element_xpath_explicity_wait_clickable(tempo, xpath)
                    self.click_elemento(elemento_label_data_partida)

                    xpath = '//*[@id="calendarContainer"]/div/div/div/div/div[2]/div[1]/div[2]'
                    elemento_proximo_mes = self.find_element_xpath_explicity_wait_clickable(5, xpath)
                    self.click_elemento(elemento_proximo_mes)
                    time.sleep(1.23)

                    # Para digitar a data de partida
                    dia_semana = self.dict_dias_semana(self._data_ida.weekday())
                    dia_partida = self._data_ida.strftime("%#d")
                    mes_partida = self._data_ida.strftime("%m")
                    ano_partida = self._data_ida.strftime("%Y")
                    mes_partida_nome = self.calcula_nome_mes(mes_partida)
                    xpath = f'[aria-label="Escolha {dia_semana}, {dia_partida} de {mes_partida_nome} de {ano_partida} como sua data de ida. Está disponível."]'
                    print(xpath)
                    elemento = WebDriverWait(self._driver, tempo).until(EC.visibility_of_element_located((By.CSS_SELECTOR, xpath)))
                    elemento.click()

                    # Para digitar a data de retorno
                    dia_semana = self.dict_dias_semana(self._data_volta.weekday())
                    dia_retorno = self._data_volta.strftime("%#d")
                    mes_retorno = self._data_volta.strftime("%m")
                    ano_retorno = self._data_volta.strftime("%Y")
                    mes_retorno_nome = self.calcula_nome_mes(mes_retorno)
                    # xpath = f'[aria-label="Escolha {dia_semana}, {dia_retorno} de {mes_retorno_nome} de {ano_retorno} como sua data de volta. Está disponível."]'
                    xpath = f'[aria-label="Escolha {dia_semana}, {dia_retorno} de {mes_retorno_nome} de {ano_retorno} como data de volta. Está disponível."]'
                    print(xpath)
                    elemento = WebDriverWait(self._driver, tempo).until(EC.element_to_be_clickable((By.CSS_SELECTOR, xpath)))
                    elemento.click()

                    # Para pesquisar os voos
                    xpath = '//*[@id="btnSearchCTA"]'
                    elemento_but_pesquisa = self.find_element_xpath_explicity_wait_clickable(tempo, xpath)
                    self.click_elemento(elemento_but_pesquisa)

                    try:
                        # Fehcar pop up
                        time.sleep(20)
                        aba = self._driver.window_handles
                        self._driver.switch_to.window(aba[0])
                        time.sleep(2)
                        self._driver.close()

                        # Ir para aba de voos
                        aba = self._driver.window_handles
                        self._driver.switch_to.window(aba[0])
                    except InvalidSessionIdException:
                        pass

                    # DataFrames que serão utilizados caso não haja passagens
                    df_sem_passagem_ida = self.cria_df_sem_passagem(origem, destino)
                    df_sem_passagem_volta = self.cria_df_sem_passagem(destino, origem)

                    # Verificar se há passagens para essa rota
                    xpath = '//*[@id="WrapperBodyFlights"]/div/ol/li[1]'  # XPATH do Primeiro voo
                    time.sleep(7)

                    try:
                        primeiro_voo = self.find_element_xpath_explicity_wait_visibility(15, xpath)
                        print('Há voos para essa rota')
                        price_final_ida = self.realizar_extracao_precos()
                        self.click_end_key()
                        time.sleep(1.5)

                        for i in range(len(price_final_ida)):
                            df_ida_juntar = pd.DataFrame(data=[origem, destino, price_final_ida[i]]).T
                            df_ida_juntar.columns = self.coluna_nomes
                            self.df_preco_ida_volta = pd.concat([self.df_preco_ida_volta, df_ida_juntar], axis=0)
                            self.df_ida_origem = pd.concat([self.df_ida_origem, df_ida_juntar], axis=0)
                        self.click_home_key()
                        time.sleep(1)
                        self.click_elemento(primeiro_voo)

                        xpath = "//*[contains(text(), 'Escolher a tarifa LIGHT e passa para o voo seguinte')]/following-sibling::span"
                        elemento_classe_economica = self.find_element_xpath_explicity_wait_clickable(tempo, xpath)
                        actions = ActionChains(self._driver)
                        actions.move_to_element(elemento_classe_economica).perform()
                        time.sleep(3)
                        self.click_elemento(elemento_classe_economica)

                        try:
                            time.sleep(5)
                            xpath = '//*[@id="WrapperBodyFlights"]/div/ol/li[1]'
                            primeiro_voo_volta = self.find_element_xpath_explicity_wait_visibility(15, xpath)
                            self.click_end_key()
                            time.sleep(1)

                            print('Há voos de volta!')
                            price_final_volta = self.realizar_extracao_precos_retorno()
                            for i in range(len(price_final_volta)):
                                df_volta_juntar = pd.DataFrame(data=[destino, origem, price_final_volta[i]]).T
                                df_volta_juntar.columns = self.coluna_nomes
                                self.df_preco_ida_volta = pd.concat([self.df_preco_ida_volta, df_volta_juntar], axis=0)

                                df_volta_juntar.columns = self.coluna_nomes_volta
                                self.df_volta_destino = pd.concat([self.df_volta_destino, df_volta_juntar], axis=0)

                        except TimeoutException:
                            print('Não há voos de volta!')
                            self.df_preco_ida_volta = pd.concat([self.df_preco_ida_volta, df_sem_passagem_volta], axis=0)
                            self.df_volta_destino = pd.concat([self.df_volta_destino, df_sem_passagem_volta], axis=0)

                    except TimeoutException:
                        print('Não há voos para essa rota')
                        self.df_preco_ida_volta = pd.concat([self.df_preco_ida_volta, df_sem_passagem_ida], axis=0)
                        self.df_ida_origem = pd.concat([self.df_ida_origem, df_sem_passagem_ida], axis=0)
                        self.df_preco_ida_volta = pd.concat([self.df_preco_ida_volta, df_sem_passagem_volta], axis=0)
                        self.df_volta_destino = pd.concat([self.df_volta_destino, df_sem_passagem_volta], axis=0)



                    print('Df de ida e volta')
                    print(self.df_preco_ida_volta)
                    print('')

                    print('Df de somente ida')
                    print(self.df_ida_origem)
                    print('')

                    print('Df de somente volta')
                    print(self.df_volta_destino)
                    print('')

                    self._driver.quit()
                    break
                except TimeoutException:
                    self._driver.quit()
                    print('Deu ruim, vai repetir de novo!')

                except ElementClickInterceptedException:
                    self._driver.quit()
                    print('Deu ruim, vai repetir de novo!')

                except InvalidSessionIdException:
                    self._driver.quit()
                    print('Deu ruim, erro de sessão inválida. \nVai repetir de novo!')

                except StaleElementReferenceException:
                    print('Deu ruim, erro de stale element reference. \nVai repetir de novo!')
        
            self.df_preco_ida_volta.to_excel(f'Extração_LATAN_{data_atual}/df_preco_latan_ida_volta_{data_atual}.xlsx')
            self.df_ida_origem.to_excel(f'Extração_LATAN_{data_atual}/df_preco_latan_ida_origem_{data_atual}.xlsx')
            self.df_volta_destino.to_excel(f'Extração_LATAN_{data_atual}/df_preco_latan_volta_destino_{data_atual}.xlsx')

        final = time.time()
        print(final - comeco)

    def cria_df_sem_passagem(self, aero1, aero2):
        data_sem_passagem = 'Sem voos disponíveis'
        df_sem_passagem = pd.DataFrame(data=[aero1, aero2, data_sem_passagem]).T
        df_sem_passagem.columns = self.coluna_nomes
        return df_sem_passagem

    def realizar_extracao_precos(self):
        html = self.retorna_html_page()
        soup = BeautifulSoup(html, 'html.parser')
        precos = soup.findAll('span', class_='sc-lmrgJh bggiiV')
        prices_final = []
        for i in range(len(precos)):
            if i % 2 == 0:
                prices_final.append(precos[i].get_text())
            else:
                pass

        return prices_final
    
    def realizar_extracao_precos_retorno(self):
        html = self.retorna_html_page()
        soup = BeautifulSoup(html, 'html.parser')
        precos = soup.findAll('span', class_='sc-lmrgJh bggiiV')
        prices_final = []
        for i in range(1, len(precos)):
            if i % 2 != 0:
                prices_final.append(precos[i].get_text())
            else:
                pass

        return prices_final

    @staticmethod
    def calcula_data_atual():
        return date.today()

    def calcula_data_ida(self):
        data_ida = self._data_atual + relativedelta(months=2)
        return data_ida

    def calcula_data_volta(self):
        data_volta = self._data_ida + relativedelta(days=8)
        return data_volta

    @staticmethod
    def calcula_nome_mes(mes_selecionado):
        month_name = {
            '01': 'janeiro',
            '02': 'fevereiro',
            '03': 'março',
            '04': 'abril',
            '05': 'maio',
            '06': 'junho',
            '07': 'julho',
            '08': 'agosto',
            '09': 'setembro',
            '10': 'outubro',
            '11': 'novembro',
            '12': 'dezembro',
        }
        return month_name.get(mes_selecionado)


def scraping_latan():
    objeto_latan = Latan()
    objeto_latan.scraping()


if __name__ == '__main__':
    scraping_latan()

Voos de FORTALEZA para VIX


[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:00<00:00, 21.7MB/s]


Cookies aceitos!
[aria-label="Escolha Sábado, 15 de abril de 2023 como sua data de ida. Está disponível."]
[aria-label="Escolha Domingo, 23 de abril de 2023 como data de volta. Está disponível."]
Há voos para essa rota
